<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 0.11.25</div>
        </div>
    </div>
</div>

# Model Parameterizations in Salvus

In this notebook we will explore all the different options to parameterize the
material properties, which are
currently supported in Salvus. As always, the first step is to import the
requisite Python modules.

In [ ]:
%matplotlib inline
import os
from functools import partial
from pathlib import Path

import h5py
import matplotlib.pyplot as plt
import numpy as np

import salvus.mesh.structured_grid_2D as sg2d
import salvus.mesh.structured_grid_3D as sg3d
import salvus.mesh.unstructured_mesh as um
import salvus.toolbox.toolbox as st
from salvus.flow import api
from salvus.flow import simple_config as sc

SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "local")

First we set up a simple function to return simple meshes for testing in both
2- and 3-D dimensions. The size of the mesh is small as these examples will
only run for a trivial duration, and are just meant to be used to explore the
possible parameterizations.

In [ ]:
def get_basic_mesh(dim: int, epd: int = 20) -> um.UnstructuredMesh:
    """Get a simple mesh to outline allowed parameter types.

    Parameters
    ----------
    dim : int
        Dimension of the mesh.
    epd : int, optional
        Elements per dimension, by default 10

    Returns
    -------
    um.UnstructuredMesh
        An unstructured mesh free of any parameters.

    """
    x = 2.0
    if dim == 2:

        mesh = sg2d.StructuredGrid2D.rectangle(
            nelem_x=epd, nelem_y=epd, max_x=x, max_y=x
        ).get_unstructured_mesh()

    elif dim == 3:

        mesh = sg3d.StructuredGrid3D.cube(
            nelem_x=epd, nelem_y=epd, nelem_z=epd, max_x=x, max_y=x, max_z=x
        ).get_unstructured_mesh()

    mesh.find_side_sets()
    mesh.change_tensor_order(4)
    return mesh

Second, we will define some basic parameter values to use (all SI units).

In [ ]:
vs = 500.0
vp = 1000.0
rho = 1000.0

Now we will define a set of sources which can be used in all of our testing
environments. We'll use a Ricker wavelet in all cases, but we'll change the
spatial type of our source depending on the dimension of the problem and the
physics.

In [ ]:
stf = sc.stf.Ricker(center_frequency=2e3)

src_scalar_2d = sc.source.cartesian.ScalarPoint2D(
    f=1, x=1, y=1, source_time_function=stf
)
src_scalar_3d = sc.source.cartesian.ScalarPoint3D(
    f=1, x=1, y=1, z=1, source_time_function=stf
)
src_vector_2d = sc.source.cartesian.VectorPoint2D(
    fx=1, fy=1, x=1, y=1, source_time_function=stf
)
src_vector_3d = sc.source.cartesian.VectorPoint3D(
    fx=1, fy=1, fz=1, x=1, y=1, z=1, source_time_function=stf
)

An finally we'll partially fill in Salvus _Flow_'s `api.run` function. We'll
be running the function many times with mostly the same setup, so this is a nice
space saving method.

In [ ]:
run_salvus = partial(
    api.run, ranks=2, get_all=True, site_name=SALVUS_FLOW_SITE_NAME
)

Now, onto the examples themselves.

## 2D domains

We'll start with all of the 2-D parameterizations. Again, to save space,
we'll prepare our `simulation.Waveform` object with values which will
be re-used.

In [ ]:
w = sc.simulation.Waveform()

w.domain.dimension = 2
w.output.volume_data.format = "hdf5"
w.output.volume_data.filename = "output.h5"
w.output.volume_data.sampling_interval_in_time_steps = 100

### Acoustic

Acoustic meshes can be parameterized either using p-velocity and density, or
a linear parameterization using M0 and M1. Acoustic elements must have the
`fluid` flag set to 1.

#### Velocity (sound speed) and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(2)

# Attach parameter to the nodes of each element.
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("VP", par_template * vp)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.ones(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["phi"]
w.physics.wave_equation.point_source = [src_scalar_2d]

# Run the solver.
output_folder = Path("acoustic_rhovp")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder)

# Visualize the results.
f, ax = plt.subplots(1, 1)
ax.set_aspect("equal")
t, da0 = st.visualize_wavefield_2d(output_file, "phi")
ax.tricontourf(t, da0[-1, :])

#### (Linear) compliance parameters

In [ ]:
# Re-parameterize as m0 and m1.
m0, m1 = (1 / rho) / vp ** 2, 1 / rho

# Generate the mesh.
m = get_basic_mesh(2)

# Attach parameter to the nodes of each element.
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("M0", par_template * m0)
m.attach_field("M1", par_template * m1)
m.attach_field("fluid", np.ones(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["phi"]
w.physics.wave_equation.point_source = [src_scalar_2d]

# Run the solver.
output_folder = Path("acoustic_linear")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder)

# Visualize the results.
f, ax = plt.subplots(1, 1)
ax.set_aspect("equal")
t, da1 = st.visualize_wavefield_2d(output_file, "phi")
ax.tricontourf(t, da1[-1, :])

# All parameterizations should have produced the same output.
np.testing.assert_allclose(da0, da1, atol=1e-3)

### Elastic

Isotropic elastic models can be parameterized using either:

* velocities {VP, VS, RHO}

* Lame's first parameter, shear modulus, and density {LAMBDA, MU, RHO}

* Bulk modulus, shear modulus, and density {KAPPA, MU, RHO}

#### Velocities and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(2)

# Attach parameter to the nodes of each element.
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("VP", par_template * vp)
m.attach_field("VS", par_template * vs)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.zeros(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["displacement"]
w.physics.wave_equation.point_source = [src_vector_2d]

# Run the solver.
output_folder = Path("elastic_vpvsrho")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder)

# Visualize the results.
f, ax = plt.subplots(1, 1)
ax.set_aspect("equal")
t, de0 = st.visualize_wavefield_2d(output_file, "displacement")
ax.tricontourf(t, de0[-1, :])

#### Lame parameters and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(2)

# Attach parameter to the nodes of each element.
mu = rho * vs ** 2
lam = rho * vp ** 2 - 2 * mu
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("LAMBDA", par_template * lam)
m.attach_field("MU", par_template * mu)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.zeros(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["displacement"]
w.physics.wave_equation.point_source = [src_vector_2d]

# Run the solver.
output_folder = Path("elastic_lambdamurho")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder)

# Visualize the results.
f, ax = plt.subplots(1, 1)
ax.set_aspect("equal")
t, de1 = st.visualize_wavefield_2d(output_file, "displacement")
ax.tricontourf(t, de1[-1, :])

#### Elastic moduli and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(2)

# Attach parameter to the nodes of each element.
mu = rho * vs ** 2
kap = rho * (vp ** 2 - 4 / 3 * vs ** 2)
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("KAPPA", par_template * kap)
m.attach_field("MU", par_template * mu)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.zeros(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["displacement"]
w.physics.wave_equation.point_source = [src_vector_2d]

# Run the solver.
output_folder = Path("elastic_kappamurho")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder)

# Visualize the results.
f, ax = plt.subplots(1, 1)
ax.set_aspect("equal")
t, de2 = st.visualize_wavefield_2d(output_file, "displacement")
ax.tricontourf(t, de2[-1, :])

# All parameterizations should have produced the same output.
np.testing.assert_allclose(de0, de1, atol=1e-7)
np.testing.assert_allclose(de1, de2, atol=1e-7)

## 3D domains
We'll generate a new partially-filled waveform simulation object for 3-D.

In [ ]:
w = sc.simulation.Waveform()

w.domain.dimension = 3
w.output.volume_data.format = "hdf5"
w.output.volume_data.filename = "output.h5"
w.output.volume_data.sampling_interval_in_time_steps = 100

### Acoustic

Acoustic meshes can be parameterized either using p-velocity and density, or
a linear parameterization using M0 and M1. Acoustic elements must have the
`fluid` flag set to 1.

#### Velocity (sounds speed) and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(3)

# Attach parameter to the nodes of each element.
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("VP", par_template * vp)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.ones(m.nelem))

# Attach the mesh.
w.set_mesh(m)
w.output.volume_data.fields = ["phi"]
w.physics.wave_equation.point_source = [src_scalar_3d]

# Run the solver.
output_folder = Path("acoustic_rhovp")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder, overwrite=True)

# Read the results.
with h5py.File(output_file, "r") as fh:
    da0 = fh["/volume/phi"][:]

#### (Linear) compliance parameters

In [ ]:
# Re-parameterize as m0 and m1.
m0, m1 = (1 / rho) / vp ** 2, 1 / rho

# Generate the mesh.
m = get_basic_mesh(3)

# Attach parameter to the nodes of each element.
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("M0", par_template * m0)
m.attach_field("M1", par_template * m1)
m.attach_field("fluid", np.ones(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["phi"]
w.physics.wave_equation.point_source = [src_scalar_3d]

# Run the solver.
output_folder = Path("acoustic_linear")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder, overwrite=True)

# Visualize the results.
with h5py.File(output_file, "r") as fh:
    da1 = fh["/volume/phi"][:]

# All parameterizations should have produced the same output.
da0_l2 = np.linalg.norm(da0)
da1_l2 = np.linalg.norm(da1)
np.testing.assert_allclose(da0_l2, da1_l2, atol=da0_l2 * 1e-6)

### Elastic

Isotropic elastic models can be parameterized using either:
* velocities {VP, VS, RHO}
* Lame's first parameter, shear modulus, and density {LAMBDA, MU, RHO}
* Bulk modulus, shear modulus, and density {KAPPA, MU, RHO}

#### Velocities and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(3)

# Attach parameter to the nodes of each element.
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("VP", par_template * vp)
m.attach_field("VS", par_template * vs)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.zeros(m.nelem))

# # Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["displacement"]
w.physics.wave_equation.point_source = [src_vector_3d]

# # Run the solver.
output_folder = Path("elastic_vpvsrho")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder, overwrite=True)

# Visualize the results.
with h5py.File(output_file, "r") as fh:
    de0 = fh["/volume/displacement"][:]

#### Lame parameters and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(3)

# Attach parameter to the nodes of each element.
mu = rho * vs ** 2
lam = rho * vp ** 2 - 2 * mu
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("LAMBDA", par_template * lam)
m.attach_field("MU", par_template * mu)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.zeros(m.nelem))

# # Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["displacement"]
w.physics.wave_equation.point_source = [src_vector_3d]

# Run the solver.
output_folder = Path("elastic_lambdamurho")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder, overwrite=True)

# # Visualize the results.
with h5py.File(output_file, "r") as fh:
    de1 = fh["/volume/displacement"][:]

#### Elastic moduli and density

In [ ]:
# Generate the mesh.
m = get_basic_mesh(3)

# Attach parameter to the nodes of each element.
mu = rho * vs ** 2
kap = rho * (vp ** 2 - (4 / 3) * vs ** 2)
par_template = np.ones_like(m.get_element_nodes()[:, :, 0])
m.attach_field("KAPPA", par_template * kap)
m.attach_field("MU", par_template * mu)
m.attach_field("RHO", par_template * rho)
m.attach_field("fluid", np.zeros(m.nelem))

# Attach the mesh and set some custom output.
w.set_mesh(m)
w.output.volume_data.fields = ["displacement"]
w.physics.wave_equation.point_source = [src_vector_3d]

# Run the solver.
output_folder = Path("elastic_kappamurho")
output_file = output_folder / "output.h5"
run_salvus(input_file=w, output_folder=output_folder, overwrite=True)

# Visualize the results.
with h5py.File(output_file, "r") as fh:
    de2 = fh["/volume/displacement"][:]

# All parameterizations should have produced the same output.
np.testing.assert_allclose(de0, de1, atol=1e-7)
np.testing.assert_allclose(de1, de2, atol=1e-7)